In [261]:
import pandas as pd
#import openpyxl

import pip
pip.main(["install", "openpyxl"])

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: openpyxl in c:\users\frede\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (3.0.10)

Requirement already satisfied: et-xmlfile in c:\users\frede\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (from openpyxl) (1.1.0)

[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: C:\Users\frede\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

In [262]:
delay_df = pd.read_excel('get_around_delay_analysis.xlsx',engine='openpyxl')
pricing_df = pd.read_csv("get_around_pricing_project.csv")

#delay_df

In [263]:
#Delay cleaning

delay_df.head()


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000.0,363965.0,mobile,canceled,NaN,NaN,NaN
1,507750.0,269550.0,mobile,ended,-81.0,NaN,NaN
2,508131.0,359049.0,connect,ended,70.0,NaN,NaN
3,508865.0,299063.0,connect,canceled,NaN,NaN,NaN
4,511440.0,313932.0,mobile,ended,NaN,NaN,NaN


In [264]:
delay_df.isnull().any()

rental_id                                     False
car_id                                        False
checkin_type                                  False
state                                         False
delay_at_checkout_in_minutes                   True
previous_ended_rental_id                       True
time_delta_with_previous_rental_in_minutes     True
dtype: bool

In [265]:
delay_df.isnull().sum()

rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64

In [266]:
delay_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  float64
 1   car_id                                      21310 non-null  float64
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(5), object(2)
memory usage: 1.1+ MB


In [267]:
delay_df.describe(include="all")

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [268]:
delay_df['ontime']=delay_df['delay_at_checkout_in_minutes'].fillna(0).apply(lambda x: 1 if x <= 0 else 0)
delay_df

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,ontime
0,505000.0,363965.0,mobile,canceled,NaN,NaN,NaN,1
1,507750.0,269550.0,mobile,ended,-81.0,NaN,NaN,1
2,508131.0,359049.0,connect,ended,70.0,NaN,NaN,0
3,508865.0,299063.0,connect,canceled,NaN,NaN,NaN,1
4,511440.0,313932.0,mobile,ended,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...
21305,573446.0,380069.0,mobile,ended,NaN,573429.0,300.0,1
21306,573790.0,341965.0,mobile,ended,-337.0,NaN,NaN,1
21307,573791.0,364890.0,mobile,ended,144.0,NaN,NaN,0
21308,574852.0,362531.0,connect,ended,-76.0,NaN,NaN,1


In [269]:
#Late checkout figures

#Late checkouts versus global checkouts
rentals_global_count = len(delay_df)
late_global = delay_df[(delay_df['delay_at_checkout_in_minutes'] > 0) & (delay_df['delay_at_checkout_in_minutes'] < 720)]
#late_global['ontime']=late_global['delay_at_checkout_in_minutes'].apply(lambda x: 1 if x <= 0 else 0)
late_global_count = len(delay_df[delay_df['delay_at_checkout_in_minutes'] > 0])
late_global_mean = delay_df[delay_df['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean()

#Late checkouts versus checkouts types (Mobile / Connect)
type_mobile_count = len(delay_df[delay_df['checkin_type'] == 'mobile'])
type_connect_count = len(delay_df[delay_df['checkin_type'] == 'connect'])

#late_type_mobile = len(delay_df[delay_df['checkin_type'] == 'connect'] & delay_df[delay_df['delay_at_checkout_in_minutes'] > 0])
#late_type_mobile = len(delay_df[delay_df['checkin_type']=='mobile'])
#late_type_connect = len(delay_df[delay_df['checkin_type'] == 'connect' & delay_df['delay_at_checkout_in_minutes'] > 0])

late_type_mobile_count = delay_df[(delay_df['checkin_type'] =='mobile') & (delay_df['delay_at_checkout_in_minutes'] > 0)]['rental_id'].count()
late_type_connect_count = delay_df[(delay_df['checkin_type'] =='connect') & (delay_df['delay_at_checkout_in_minutes'] > 0)]['rental_id'].count()

#late_global_mean
late_global
#rentals_numbers

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,ontime
2,508131.0,359049.0,connect,ended,70.0,NaN,NaN,0
9,513434.0,256528.0,connect,ended,23.0,NaN,NaN,0
12,514257.0,402596.0,mobile,ended,214.0,NaN,NaN,0
13,515147.0,257466.0,mobile,ended,15.0,NaN,NaN,0
17,518895.0,398816.0,mobile,ended,66.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...
21299,571738.0,406927.0,mobile,ended,68.0,NaN,NaN,0
21301,572736.0,393486.0,mobile,ended,180.0,NaN,NaN,0
21303,573305.0,392752.0,mobile,ended,72.0,NaN,NaN,0
21307,573791.0,364890.0,mobile,ended,144.0,NaN,NaN,0


In [270]:
import math

#Cancellations number vs global cases
cancellations = len(delay_df[delay_df['state'] == 'canceled'])
#cancellations

rental_chain = delay_df.dropna(subset=['previous_ended_rental_id'])
#rental_chain = rental_chain[rental_chain['delay_at_chekout'] == 'canceled']
cancellations_after_booking = len(rental_chain[rental_chain['state'] == 'canceled'])
cancellations_after_booking
#cancellations_after_booking = delay_df[(delay_df[delay_df['state'] == 'canceled']) & (~delay_df['previous_ended_rental_id'].isnull())].count()

229

In [271]:
rentals_historical = pd.merge(delay_df, delay_df, how='inner', left_on = 'previous_ended_rental_id', right_on = 'rental_id')
rentals_historical.head()

,rental_id_x,car_id_x,checkin_type_x,state_x,delay_at_checkout_in_minutes_x,previous_ended_rental_id_x,time_delta_with_previous_rental_in_minutes_x,ontime_x,rental_id_y,car_id_y,checkin_type_y,state_y,delay_at_checkout_in_minutes_y,previous_ended_rental_id_y,time_delta_with_previous_rental_in_minutes_y,ontime_y
0,511639.0,370585.0,connect,ended,-15.0,563782.0,570.0,1,563782.0,370585.0,connect,ended,136.0,NaN,NaN,0
1,519491.0,312389.0,mobile,ended,58.0,545639.0,420.0,0,545639.0,312389.0,mobile,ended,140.0,NaN,NaN,0
2,521156.0,392479.0,mobile,ended,NaN,537298.0,0.0,1,537298.0,392479.0,mobile,ended,NaN,NaN,NaN,1
3,525044.0,349751.0,mobile,ended,NaN,510607.0,60.0,1,510607.0,349751.0,mobile,ended,-113.0,NaN,NaN,1
4,528808.0,181625.0,connect,ended,-76.0,557404.0,330.0,1,557404.0,181625.0,connect,ended,-352.0,554077.0,120.0,1


In [272]:
rentals_historical.sort_values(by=['car_id_x'])
rentals_historical = rentals_historical[rentals_historical['state_x']=='canceled']
rentals_historical = rentals_historical[rentals_historical['delay_at_checkout_in_minutes_y'] > 0]
rentals_historical = rentals_historical[rentals_historical['delay_at_checkout_in_minutes_y'] > rentals_historical['time_delta_with_previous_rental_in_minutes_x']]
rentals_historical['rental_id_x'].count()

37

In [273]:
rentals_historical.drop(["car_id_y"], axis=1)
rentals_historical.rename(columns = {'rental_id_x':'rental_id','car_id_x':'car_id','checkin_type_x':'checkin_type_before','state_x':'state_before'}, inplace = True)
rentals_historical.sort_values(by=['car_id'])
rentals_historical

,rental_id,car_id,checkin_type_before,state_before,delay_at_checkout_in_minutes_x,previous_ended_rental_id_x,time_delta_with_previous_rental_in_minutes_x,ontime_x,rental_id_y,car_id_y,checkin_type_y,state_y,delay_at_checkout_in_minutes_y,previous_ended_rental_id_y,time_delta_with_previous_rental_in_minutes_y,ontime_y
127,539151.0,282893.0,mobile,canceled,NaN,548646.0,30.0,1,548646.0,282893.0,mobile,ended,201.0,NaN,NaN,0
154,553139.0,297851.0,mobile,canceled,NaN,547650.0,240.0,1,547650.0,297851.0,mobile,ended,410.0,NaN,NaN,0
170,559037.0,349171.0,connect,canceled,NaN,552392.0,0.0,1,552392.0,349171.0,connect,ended,46.0,NaN,NaN,0
184,543745.0,359045.0,connect,canceled,NaN,521852.0,150.0,1,521852.0,359045.0,connect,ended,650.0,NaN,NaN,0
205,538224.0,340014.0,connect,canceled,NaN,534999.0,180.0,1,534999.0,340014.0,connect,ended,346.0,NaN,NaN,0
214,551372.0,261576.0,mobile,canceled,NaN,547240.0,210.0,1,547240.0,261576.0,mobile,ended,550.0,NaN,NaN,0
307,568105.0,351546.0,connect,canceled,NaN,564144.0,120.0,1,564144.0,351546.0,connect,ended,210.0,NaN,NaN,0
363,546930.0,316449.0,mobile,canceled,NaN,551866.0,0.0,1,551866.0,316449.0,mobile,ended,153.0,NaN,NaN,0
491,561559.0,390927.0,connect,canceled,NaN,537960.0,0.0,1,537960.0,390927.0,connect,ended,31.0,NaN,NaN,0
565,556126.0,396250.0,mobile,canceled,NaN,548864.0,510.0,1,548864.0,396250.0,mobile,ended,527.0,550060.0,120.0,0


Pricing Analyzis

In [274]:
pricing_df.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [275]:
pricing_df= pricing_df.drop("Unnamed: 0", axis=1)
pricing_df.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [276]:
pricing_df.describe(include="all")

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [277]:
print("Percentage of missing values: ")
display(100*pricing_df.isnull().sum()/pricing_df.shape[0])

Percentage of missing values: 


model_key                    0.0
mileage                      0.0
engine_power                 0.0
fuel                         0.0
paint_color                  0.0
car_type                     0.0
private_parking_available    0.0
has_gps                      0.0
has_air_conditioning         0.0
automatic_car                0.0
has_getaround_connect        0.0
has_speed_regulator          0.0
winter_tires                 0.0
rental_price_per_day         0.0
dtype: float64

In [278]:
px.scatter(pricing_df, x = 'mileage', y = 'rental_price_per_day')

In [279]:
px.scatter(pricing_df, x = 'engine_power', y = 'rental_price_per_day')

In [280]:
px.scatter(pricing_df, x = 'car_type', y = 'rental_price_per_day')

Machine learning part

In [281]:
pricing_df.columns

Index(['model_key', 'mileage', 'engine_power', 'fuel', 'paint_color',
       'car_type', 'private_parking_available', 'has_gps',
       'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
       'has_speed_regulator', 'winter_tires', 'rental_price_per_day'],
      dtype='object')

In [282]:
X = pricing_df.iloc[:, :-1]
y = pricing_df.iloc[:, -1:]


In [283]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Dividing into train and test sets...


In [284]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['mileage', 'engine_power']
Found categorical features  ['model_key', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']


In [285]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [286]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[      
        ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
        ('encoder', OneHotEncoder(drop='first',handle_unknown = 'ignore')) # first column will be dropped to avoid creating correlations between features
    ])

In [287]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [288]:
# Instantiating a linear regression model
model = Pipeline(steps=[
("Preprocessing", preprocessor),
("Regressor", LinearRegression())
])

model.fit(X_train, y_train)

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['model_key', 'fuel',
                                                   'paint_color', 'car_type',
                                                   'private_parking_available',
                                                   'has_gps',
                                                   'has_air_conditioning',
                                                   'automatic_car',
                                                   'has_getaround_connect',
                                                   'has_speed_regulator',
                                                   'winter_tires'])])),
                ('Regressor', LinearRegression())])

In [289]:
# Train model
print("Train model...")
model.fit(X_train, y_train) # Training is always done on train set !!
print("...Done.")

Train model...
...Done.


In [290]:
# Predictions on test set
print("Predictions on test set...")
y_test_pred = model.predict(X_test)
print("...Done.")
print(y_test_pred)
print()

Predictions on test set...
...Done.
[[  91.36917976]
 [ 157.56523373]
 [ 105.17951053]
 [  72.22062452]
 [ 100.58731818]
 [ 173.65692543]
 [ 127.63074732]
 [ 188.7341918 ]
 [ 163.41184374]
 [ 114.06701325]
 [ 103.18724204]
 [  97.99945849]
 [ 115.01639754]
 [ 115.78221275]
 [ 188.40550943]
 [ 126.18644916]
 [  93.67103921]
 [ 189.41604073]
 [ 121.30449676]
 [ 110.29831572]
 [ 196.26972758]
 [ 126.05809288]
 [ 110.71844383]
 [ 126.91982812]
 [ 116.90378425]
 [ 118.48905309]
 [ 122.71971673]
 [ 105.90671162]
 [  95.21517359]
 [ 127.52217965]
 [  24.19959672]
 [ 106.6504364 ]
 [ 124.77471705]
 [ 112.70275559]
 [ 138.04729223]
 [ 131.13294331]
 [  89.31821412]
 [ 103.49156005]
 [ 110.60140135]
 [  87.05382389]
 [ 136.47733117]
 [ 113.81151754]
 [  88.43603076]
 [ 124.92212242]
 [  96.4531336 ]
 [ 170.06411439]
 [ 109.09035978]
 [ 145.12989844]
 [ 109.68884307]
 [ 163.16834544]
 [ 150.86417326]
 [  90.38638373]
 [ 123.47067235]
 [ 132.24175737]
 [ 176.34438471]
 [ 141.89995731]
 [ 119.97224

C:\Users\frede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros



In [291]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(y_train, y_train_pred))
print("R2 score on test set : ", r2_score(y_test, y_test_pred))

R2 score on training set :  0.715670343584498
R2 score on test set :  0.6892822444095184


In [293]:
predict = model.predict(X_train)

In [ ]:
import pickle

pickle.dump(model,'./model','wb')

In [294]:
from joblib import dump, load


#dumping models
dump(preprocessor, "./preprocessor.joblib")
dump(model, "./model.joblib")

['./model.joblib']

In [295]:
model_job_load = load('model.joblib')
model_job_load.predict(X_test)

C:\Users\frede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros



array([[  91.36917976],
       [ 157.56523373],
       [ 105.17951053],
       [  72.22062452],
       [ 100.58731818],
       [ 173.65692543],
       [ 127.63074732],
       [ 188.7341918 ],
       [ 163.41184374],
       [ 114.06701325],
       [ 103.18724204],
       [  97.99945849],
       [ 115.01639754],
       [ 115.78221275],
       [ 188.40550943],
       [ 126.18644916],
       [  93.67103921],
       [ 189.41604073],
       [ 121.30449676],
       [ 110.29831572],
       [ 196.26972758],
       [ 126.05809288],
       [ 110.71844383],
       [ 126.91982812],
       [ 116.90378425],
       [ 118.48905309],
       [ 122.71971673],
       [ 105.90671162],
       [  95.21517359],
       [ 127.52217965],
       [  24.19959672],
       [ 106.6504364 ],
       [ 124.77471705],
       [ 112.70275559],
       [ 138.04729223],
       [ 131.13294331],
       [  89.31821412],
       [ 103.49156005],
       [ 110.60140135],
       [  87.05382389],
       [ 136.47733117],
       [ 113.811